Imports

In [350]:
import gurobipy as gp
from gurobipy import *
import numpy as np
import pandas as pd

from db_manager import *
from load_data import *
from printing import *

In [351]:
# input = "input.xlsm"
input = "V:\\Horarios 2025\\2do semestre\\input.xlsx"

output = "output.xlsx"

In [352]:
# clases
from entities import *
from variables import *

# Formulacion

## (1) Datos / Constantes

### Horarios

In [353]:


dias = [
    Dia(1, "lun"),
    Dia(2, "mar"),
    Dia(3, "mie"),
    Dia(4, "jue"),
    Dia(5, "vie")
]
dias_ids = [d.id for d in dias]

DIAS = dias

In [354]:
horarios: list[Horario] = []

for inicio, fin, turnos, turnos_excepcional in read_horarios(get_database_connection()):
    # print(f"Creando horario de {inicio} a {fin} con turnos {turnos}")
    add_horario(horarios, inicio.strftime("%H:%M"), fin.strftime("%H:%M"), turnos, turnos_excepcional)


horarios_ids = [h.id for h in horarios]

HORARIOS = horarios

turnos = list(set([t for h in horarios for t in h.turnos]))

bloques_horario: dict[tuple, BloqueHorario] = {}

for dia in dias:
    for horario in horarios:
        bloques_horario[(dia.id, horario.id)] = BloqueHorario(dia, horario)

bloques_horario_ids = [b for b in bloques_horario]

BLOQUES_HORARIO = bloques_horario


In [355]:
# imprimir horarios en excel

# data_horarios = {
#         '#': range(1, len(horarios) + 1),
#         'Inicio': [h.inicio for h in horarios],
#         'Fin': [h.fin for h in horarios],
#         'Turnos': [", ".join(h.turnos) for h in horarios]
# }

# with pd.ExcelWriter(output) as writer:
#     pd.DataFrame(data_horarios).to_excel(writer, sheet_name="turnos", index=False)

### Profesores

In [356]:
# traer de excel

profesores: list[Profesor] = []

df = pd.read_excel(input, sheet_name="profesores")

for i in range(len(df)):
    nombre = df["Profesor"][i]
    nombre_completo = df["Nombre"][i]
    id = df["ID"][i]
    cedula = df["Cedula"][i]
    mail = df["Mail"][i]

    min_max_dias = df["Min/Max dias"][i]
    if min_max_dias not in ["min", "max"]:
        min_max_dias = None
        
    if isinstance(nombre, str):
        add_profesor(profesores, int(id), nombre, min_max_dias, nombre_completo, cedula, mail)

profesores_ids = []
for p in profesores:
    profesores_ids.append(p.id)

PROFESORES = profesores


c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [357]:
# # traer de base de datos

# profesores = []

# for nombre, nombre_completo, min_max_dias, prioridades in read_profesores(get_database_connection()):

#     add_profesor(profesores, nombre, min_max_dias, nombre_completo)
    
#     profesores[-1].set_prioridades(bloques_horario, [
#         [[id, int(valor)] for id, b in bloques_horario.items() if b.dia.nombre == dia and b.horario.inicio == hora_inicio][0]
#         for dia, hora_inicio, valor in prioridades
#     ])

# profesores_ids = [p.id for p in profesores]

# PROFESORES = profesores


In [358]:
# prioridades fijas

for p in profesores:
    p.set_prioridades(bloques_horario, fixed_pr(bloques_horario, 1)) # prioridad = 1 para todos los casos
    

In [359]:
print(*[p for p in profesores], sep=", ")

as, ab, ap, ac, ape, am, ame, api, apa, bt, ca, cb, cp, cr, dc, dj, da, ds, dl, dv, el, ep, fn, fr, gc, gb, gba, gr, gs, gco, gp, hr, ig, jb, jp, jc, jd, jg, jcc, jdm, jk, jpf, jpe, jpa, lg, ma, mb, mf, mp, mz, mr, mjc, mpe, mg, mbo, mpr, mri, md, nd, nn, pn, pp, ps, sb, sp, svv, sgp, sv, vr, wc


### Grupos

In [360]:
#grupos
"""
This script processes an Excel file containing group information and populates a list of group objects.
Functions:
    add_grupo(grupos, *args): Adds a group to the list of groups.
Variables:
    grupos (list): A list to store group objects.
    df (DataFrame): DataFrame containing the data read from the Excel file.
    grupos_ids (list): A list to store the IDs of the groups.
Workflow:
1. Read the Excel file input from the sheet named "grupos" into a DataFrame.
2. Iterate over each row in the DataFrame and extract relevant information.
3. Append extracted information to a list of arguments.
4. Replace 'nan' values with None and attempt to convert arguments to integers.
5. Call the add_grupo function with the list of arguments to add a group to the grupos list.
6. Populate the grupos_ids list with the IDs of the groups.
"""
grupos: list[Grupo] = []


df = pd.read_excel(input, sheet_name="grupos")

for i in range(len(df)):
    
    args = []
    args.append(df["Año"][i])  # anio
    args.append(df["Turno"][i])     # turno
    args.append(df["Carrera"][i])   # carrera
    args.append(df["Particion"][i])  # particion
    args.append(isinstance(df["Recurse"][i], str)) # recurse

    for i in range(0, len(args)):
        
        if str(args[i]) == "nan":
            args[i] = None

        try:
            args[i] = int(args[i])
        except:
            pass

    add_grupo(grupos, *args)

grupos_ids = [g.id for g in grupos]

GRUPOS = grupos

In [361]:
print(*[str(g) for g in grupos])

11 12 13 1REC1 1REC2 1LINF 2CIV 2IND 2TEL 2INF 2DIA 2LINF 3CIV 3IND 3TEL 3INF1 3INF2 3LINF 4CIV 4IND 4TEL 4INF 4LINF 5CIV 5IND 5TEL 5INF


In [362]:
anios = list(set([g.anio for g in grupos if g.anio != None]))  # anios

ANIOS = anios


### Materias 

In [363]:
#materias
"""
This script processes an Excel file containing information about various subjects (materias) and extracts relevant details to create a list of subjects.
The script performs the following steps:
1. Reads the Excel file input from the sheet named "materias".
2. Iterates through each row of the DataFrame to extract subject details.
3. For each subject, it extracts the name, hourly load, number of days per week, and the number of professors.
4. It then attempts to extract the names of the professors and groups associated with each subject.
5. Finally, it adds the subject to the list of subjects using the `add_materia` function.
Variables:
- materias: A list to store the details of each subject.
- df: DataFrame containing the data read from the Excel file.
- nombre: The name of the subject.
- carga_horaria: The hourly load of the subject.
- cantidad_dias: The number of days per week the subject is taught.
- cantidad_profesores: The number of professors teaching the subject.
- profs: A list to store the names of the professors.
- gs: A list to store the names of the groups.
Functions:
- add_materia: Adds a subject to the list of subjects.
- lista_grupos: Processes and returns a list of groups.
- lista_profesores: Processes and returns a list of professors.
"""

materias: list[Materia] = []

df = pd.read_excel(input, sheet_name="materias")

# print(df)

for i in range(len(df)):

    nombre = df["Materia"][i]
    nombre_completo = df["Nombre completo"][i]
    if not isinstance(nombre, str):
        continue # cancel if no name is provided

    id = df["ID"][i]

    carga_horaria = df["Carga Horaria"][i]
    cantidad_dias = df["Días por semana"][i]
    electiva = isinstance(df["Electiva"][i], str)
    teo_prac = df["Teo/ Prac"][i]
    if teo_prac not in ["teo", "prac"]:
        teo_prac = None
    
    try:
        cantidad_profesores = int(df["Cantidad profesores"][i])
    except:
        cantidad_profesores = 1 # default value

    profs = []
    p = 1
    while True:
        try:
            new_p = df["Profesor " + str(p)][i]
            if isinstance(new_p, str):
                profs.append(new_p)
            p += 1
        except:
            break

    gs = []
    g = 1
    while True:
        try:
            new_g = df["Grupo " + str(g)][i]
            if isinstance(new_g, str):
                gs.append(new_g)
            g += 1
        except:
            break

    
    
    # gs = lista_grupos(grupos, [x.strip() for x in str(df["Grupos"][i]).split(',')])
    # profs = lista_profesores(profesores, [x.strip() for x in str(df["Profesores"][i]).split(',')])

    add_materia(materias, int(id), nombre, nombre_completo, carga_horaria, cantidad_dias, lista_grupos(grupos, gs), lista_profesores(profesores, profs), cantidad_profesores, electiva, teo_prac)


materias_ids = [m.id for m in materias]

MATERIAS = materias


c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [364]:
# # print(*[f"{m}: {[str(p) for p in m.profesores]}" for m in materias], sep="\n")

# for p in profesores:
#     print(f"{p}: {[str(m) for m in materias if p in m.profesores]}")

In [365]:
# materias por profesor:
"""
This script processes an Excel file containing information about professors and their respective subjects.
It reads the data from the specified sheet and updates the list of subjects for each professor.
Steps:
1. Load the Excel file "input.xlsm" and read the sheet named "profesores" into a DataFrame.
2. Iterate through each row of the DataFrame to extract professor names and their subjects.
3. For each professor, gather the subjects they teach along with the maximum number of groups for each subject.
4. Update the professor's list of subjects using the `search_profesor_by_nombre` function.
Variables:
- df: DataFrame containing the data from the Excel sheet.
- nombre: Name of the professor.
- lista_mats: List of dictionaries, each containing the subject name and the maximum number of groups.
- m: Counter for the subject columns in the DataFrame.
- new_m: Name of the subject being processed.
- grupos_max: Maximum number of groups for the subject.
- prof: Professor object found using the `search_profesor_by_nombre` function.
Functions:
- search_profesor_by_nombre(profesores, nombre): Searches for a professor by name in the given list of professors.
Note:
- The script assumes that the Excel file and sheet structure are consistent with the expected format.
- The `search_profesor_by_nombre` function and other commented-out functions need to be defined elsewhere in the codebase.
"""
df = pd.read_excel(input, sheet_name="profesores")

for i in range(len(df)):
    nombre = df["Profesor"][i]

    lista_mats = []
    m = 1
    while True:
        try:
            new_m = df["Materia " + str(m)][i]
            
            if isinstance(new_m, str):
                grupos_max = int(df["Grupos materia " + str(m)][i])
                lista_mats.append({"nombre_materia" : new_m, "grupos_max" : grupos_max})
            m += 1
        except:
            break
    
    prof = search_profesor_by_nombre(profesores, nombre)
    if not prof is None:
        prof.lista_materias = lista_mats


    # add_materia(materias, id, nombre, carga_horaria, cantidad_dias, lista_grupos(grupos, gs), lista_profesores(profesores, profs))


c:\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [366]:
# for p in profesores:
#     print(f"{p}: {[str(m) for m in materias if p in m.profesores]} =? {[f"{m['nombre_materia'], m['grupos_max']}" for m in p.lista_materias]}")

In [367]:
## cargar a la base de datos
connection = get_database_connection()
write_profesores(connection, profesores)
write_materias(connection, materias, profesores)


### Superposicion

In [368]:
# superposicion
"""
Calculates the overlap between pairs of subjects and stores the results in a dictionary.

This script iterates over all pairs of subjects in the `materias` list and calculates the overlap 
between each pair using the `calcular_super` function. The results are stored in the `superposicion` 
dictionary, where the keys are tuples of subject IDs and the values are the calculated overlaps.

Variables:
    superposicion (dict): A dictionary to store the overlap results between pairs of subjects.
    materias (list): A list of subject objects, each with an `id` attribute.

Functions:
    calcular_super(m1, m2): A function that takes two subject objects and returns the overlap between them.

Dictionary Structure:
    superposicion[(m1.id, m2.id)] = overlap_value
"""
superposicion: dict[tuple, Superposicion] = {}

for m1 in materias:
    for m2 in materias:
        superposicion[(m1.id, m2.id)] = calcular_super(m1, m2)



In [369]:

# CASO PARTICULAR: permitir superposicion de Estadistica Aplicada con Metodologia de la Investigacion
fix_super(0, superposicion,
            [m for m in materias if m.nombre == "EA"],
            [m for m in materias if m.nombre == "MInv"]
            )


### Electivas

Fijar superposicion = 0 (permitir superposicion)

In [370]:
for e in electivas(materias):
    fix_super(0, superposicion, [e], electivas(materias))

SUPERPOSICION = superposicion

In [371]:
superposicion_electivas: dict[tuple, Superposicion] = {}

for m1 in electivas(materias):
    for m2 in electivas(materias):
        superposicion_electivas[(m1.id, m2.id)] = calcular_super(m1, m2)

In [372]:
# automatizar:
# df = pd.read_excel(input, sheet_name="electivas")

# for r in range(len(df)):

#     try:
#         mat1 = search_materias_by_nombre(materias, str(df["Materia"][r]))[0]
#     except:
#         continue

#     for c in df.columns:

#         try:
#             mat2 = search_materias_by_nombre(materias, str(c))[0]
            
#             if isinstance(df[c][r], str):
#                 superposicion[(mat1.id, mat2.id)] = Superposicion(0, mat1, mat2)
#                 print(mat1, mat2)
#         except:
#             continue



In [373]:

# permitir superposicion de electivas entre si
# fix_super(0, superposicion_electivas, search_materias_by_nombre(electivas(materias), "AgroN"), electivas(materias))


SUPERPOSICION_ELECTIVAS = superposicion_electivas

In [374]:
print("Grupos:", len(grupos))
# print([str(i) for i in grupos])
print("Materias:", len(materias))
# print([str(i) for i in materias])
print("Profesores:", len(profesores))
# print([str(i) for i in profesores])

Grupos: 27
Materias: 116
Profesores: 70


## (2) Variables


$$ u_{mb} \in \{0,1\} $$

    m: materia
    b: bloque horario = [dh]: dia y hora

$$ v_{md} \in \{0,1\} $$

    m: materia
    d: dia

$$ w_{mp} \in \{0,1\} $$

    m: materia
    p: profesor


$$ x_{gb} \in \{0,1\} $$

    g: grupo
    b: bloque horario = [dh]: dia y hora

$$ y_{pb} \in \{0,1\} $$

    p: profesor
    b: bloque horario = [dh]: dia y hora

$$ z_{pd} \in \{0,1\} $$

    p: profesor
    d: dia

In [375]:
print(len(materias))
print(len(bloques_horario))

116
80


## (3) Restricciones

### (3.1) Materias

#### (3.1.1) superposicion (redundante con la definicion de la variable x)
$$ \sum_{b} \sum_{m, m'}{u_{mb} \times S_{m,m'} \times u_{m'b}} = 0 $$


In [376]:
"""
    This code defines constraints to ensure that no two subjects (materias) overlap
    in the same time block (bloque horario).
    Constraints:
        1. For each time block (b) in bloques_horario_ids, the sum of the product of the decision variables 
        (u_dict[m1, b].variable and u_dict[m2, b].variable) and the overlap value (superposicion[(m1, m2)].value)
        for all pairs of subjects (m1, m2) must be equal to 0.
"""

def constr_superposicion(model, u_dict):

    model.addConstrs(gp.quicksum(u_dict[m1, b].variable * superposicion[(m1, m2)].value * u_dict[m2, b].variable
                            for m1 in materias_ids for m2 in materias_ids)
                == 0 for b in bloques_horario_ids)


## for g in grupos:
##     mats = materias_grupo(g, materias)
##     if len(mats) > 1:
##         # model.addConstrs(gp.quicksum(u_dict[m.id, b].variable for m in mats) <= 1 for b in bloques_horario_ids)
##         model.addConstrs(gp.quicksum(u_dict[m.id, b].variable for m in mats) <= 1 for b in bloques_horario_ids)


In [377]:
# print(*[[str(m), m.cantidad_dias] for m in materias], sep="\n")

#### (3.1.2) cubrir carga horaria para cada materia
$$ \sum_b{u_{mb}} = C_m $$

    para todo m (c_m: carga horaria)

In [378]:
def constr_carga_horaria(model, u_dict):
    for m in materias:
        model.addConstr(gp.quicksum(u_dict[m.id, b].variable for b in bloques_horario_ids) == m.carga_horaria)

#### (3.1.3) particion de horas por materia

##### (3.1.3.1) fijar cantidad de dias por materia

$$ \sum_d {v_{md}} = D_m $$

    para todo m

In [379]:
def constr_dias_materia(model, v_dict):
    for m in materias:
        model.addConstr(gp.quicksum(v_dict[m.id, d].variable for d in dias_ids) == m.cantidad_dias)

##### (3.1.3.2) fijar maximo y minimo de horas por dia

$$ v_{md} \times {H_{MIN}}_m \leq \sum_h {u_{mdh}} \leq v_{md} \times {H_{MAX}}_m $$

    para todo m, d

In [380]:
def constr_max_min_horas(model, u_dict, v_dict):

    for m in materias:

        model.addConstrs(gp.quicksum(u_dict[(m.id,(d,h))].variable for h in horarios_ids)
                        <= m.horas_max() * v_dict[m.id,d].variable for d in dias_ids)
        
        model.addConstrs(gp.quicksum(u_dict[(m.id,(d,h))].variable for h in horarios_ids)
                        >= m.horas_min() * v_dict[m.id,d].variable for d in dias_ids)
    

##### (3.1.3.3) fijar materia a turno de horarios

$$ u_{mb} = 0 $$

    para todo b fuera de m.turnos
    para todo m



In [381]:
def constr_turnos_materia(model, u_dict):

    for m in materias:
        no_bloques_materia_ids = [b for b in bloques_horario if b not in bloques_horario_materia(m, bloques_horario)]
        # for b in bloques_horario:
        #     bloques_horario_materia(m, bloques_horario)
        #     if len([t for t in bloques_horario[b].horario.turnos if t in m.turnos()]) == 0:
        #         no_bloques_materia_ids.append(b)
        model.addConstrs(u_dict[m.id, b].variable == 0 for b in no_bloques_materia_ids)


#### (3.1.4) horas consecutivas dentro de un dia

$$ \sum_h {u_{mdh}} - \sum_h {u_{md(h)} · u_{md(h+1)}} = v_{md} $$
    para todo m, d

In [382]:
def constr_horas_consecutivas(model, u_dict, v_dict):

    model.addConstrs(gp.quicksum(u_dict[(m,(d,h))].variable for h in horarios_ids)
                 - gp.quicksum(u_dict[(m,(d,h))].variable * u_dict[(m,(d,h+1))].variable for h in horarios_ids[0:-1])
                 == v_dict[m, d].variable for m in materias_ids for d in dias_ids)

Alternativa:
$$ \sum_h {u_{mdh}} - \sum_h {u_{md(h)} · u_{md(h+1)}} \leq 1 $$
    para todo m, d

#### (3.1.5) evitar dias consecutivos para una misma materia

$$ \sum_m \sum_d {v_{m(d)} · v_{m(d+1)}} = 0 $$


In [383]:
def constr_dias_consecutivos(model, v_dict):
    model.addConstrs(gp.quicksum(v_dict[m,d].variable * v_dict[m,d+1].variable for d in dias_ids[0:-1]) == 0
                #  for m in materias_ids)
                 for m in [mat.id for mat in materias if not mat.electiva])


#### (3.1.6) indisponibilidad de materia (opcional)

$$ u_{mb} = 0 $$

    para todo m sin disponibilidad en b

In [384]:
def constr_no_disponible_materia(model, u_dict):
    for m in materias:
        for b in m.no_disponible:
            model.addConstr(u_dict[m.id, b.id].variable == 0)

### (3.2) Profesores

#### (3.2.1) indisponibilidad

$$ \sum_m w_{mp} \times u_{mb} = 0 $$

    para todo p sin disponibilidad en b

In [385]:
def constr_no_disponible_profesor(model, u_dict, w_dict):
    for m in materias:
        for p in m.profesores:
            for b in p.no_disponible:
                model.addConstr(u_dict[m.id, b.id].variable * w_dict[m.id, p.id].variable == 0)

LINEALIZAR: SUSTITUIR CON VARIABLES y_pb

$$ y_{pb} = 0 $$

    para todo p sin disponibilidad en b

In [386]:
def constr_no_disponible_profesor_lineal(model, y_dict):
    for p in profesores:
        for b in p.no_disponible:
            model.addConstr(y_dict[p.id, b.id].variable == 0)

#### (3.2.2) unica materia por profesor para un mismo bloque horario

$$ \sum_{m} {w_{mp} \times u_{mb}} \leq 1 $$

    para todo b, p

REDUNDANTE CON DEFINICION DE y_pb

In [387]:
def constr_unica_materia_profesor(model, u_dict, w_dict):

    p_grupos_simultaneos = ["rs", "er", "dd", "ns", "jl", "fb"]

    for p in [p for p in profesores if p.nombre not in p_grupos_simultaneos]:
        model.addConstrs(gp.quicksum(u_dict[m, b].variable * w_dict[m, p.id].variable for m in materias_ids)
                     <= 1 for b in bloques_horario_ids)
        
    # caso particular
    for p in [p for p in profesores if p.nombre in p_grupos_simultaneos]:
        model.addConstrs(gp.quicksum(u_dict[m, b].variable * w_dict[m, p.id].variable for m in materias_ids)
                     <= 2 for b in bloques_horario_ids)

#### (3.2.3) profesores por materia


##### (3.2.3.1) limitar profesores a lista

$$ w_{mp} = 0 $$

    para todo p not in m.lista_profesores
    para todo m

In [388]:
def constr_limitar_profesores_materia(model, w_dict):
    
    for m in materias:
        for p in profesores:
            if p not in m.profesores:
                model.addConstr(w_dict[m.id, p.id].variable == 0)

##### (3.2.3.2) cantidad de profesores para una misma materia

$$ \sum_p {w_{mp}} = P_m $$

    para todo m

In [389]:
def constr_cantidad_profesores(model, w_dict):
    for m in materias:
        model.addConstr(gp.quicksum(w_dict[m.id, p].variable for p in profesores_ids) == m.cantidad_profesores)

#### (3.2.4) carga horaria por docente: limitar la cantidad de materias por profesor


$$ \sum_m {w_{mp}} \leq K_p $$

    para todo p



In [390]:
# separar por nombre de materia
def constr_grupos_max_profesor(model, w_dict):
    for p in profesores:
        for l in p.lista_materias:
            mats = search_materias_by_nombre(materias, l["nombre_materia"])
            if len(mats) > 0:
                model.addConstr(gp.quicksum(w_dict[m.id, p.id].variable for m in mats) <= l["grupos_max"])
        

#### (3.2.5) definicion de variable "y" (asignacion horario-profesor)

$$ y_{pb} = \sum_m{u_{mb}·w_{mp}} $$

$$ y_{pb} = \text{OR}_m \{u_{mb}·w_{mp}\} $$

    para todo p, b

In [391]:
def constr_definir_y(model, y_dict, u_dict, w_dict):

    for p in profesores:
        uw_vars = {} # variable auxiliar = u_mb·w_mp
        mats_p = materias_profesor(p, materias)
        for m in mats_p:
            # for b_id in bloques_horario_materia(m, bloques_horario):
            for b_id in bloques_horario_ids:
                uw_vars[m.id,b_id] = model.addVar(vtype=GRB.INTEGER, name=str(m)+str(b_id)+str(p)+"_uw")
                model.addConstr(uw_vars[m.id,b_id] == u_dict[m.id,b_id].variable * w_dict[m.id,p.id].variable)
        
        mats_p_ids = [m.id for m in mats_p]
        model.addConstrs(y_dict[p.id,b_id].variable ==
                     gp.or_(uw_vars[m_id,b_id] for m_id in mats_p_ids)
                     for b_id in bloques_horario_ids)
    
    # model.addConstrs(y_dict[p, b].variable ==
    #              gp.quicksum(u_dict[m,b].variable * w_dict[m,p].variable for m in materias_ids)
    #              for b in bloques_horario_ids for p in profesores_ids)

#### (3.2.6) definicion de variable "z" (para dias con clase por profesor)

$$ z_{pd} = OR_h{y_{p(dh)}} $$

    para todo p, d

In [392]:
def constr_definir_z(model, z_dict, y_dict):
    
    model.addConstrs(z_dict[p,d].variable ==
                gp.or_(y_dict[p,(d,h)].variable for h in horarios_ids)
                for p in profesores_ids for d in dias_ids)

Posible linealizacion:

$$ z_{pd} \leq \sum_h {y_{pdh}} \leq M \times z_{pd} $$

    para todo m, d


In [393]:
# for p in profesores:

#     model.addConstrs(gp.quicksum(y_dict[p.id, (d,h)].variable for h in horarios_ids)
#                      <= 10 * z_dict[p.id,d].variable for d in dias_ids)
    
#     model.addConstrs(gp.quicksum(y_dict[p.id, (d,h)].variable for h in horarios_ids)
#                      >= z_dict[p.id,d].variable for d in dias_ids)

#### (3.2.7) limitar la cantidad de horas diarias por profesor (NO VA)


$$ \sum_{m,h} {w_{mp}·u_{mdh}} \leq {HD}_p $$

    para todo p, d


In [394]:
# FALTA CHEQUEAR

# for p in profesores:
#     if not p.horas_max is None:
#         # mats = [search_materias_by_nombre(materias, m).id for m in p.materias()]
#         model.addConstrs(gp.quicksum(
#             w_dict[m, p.id].variable * u_dict[m,(d,h)].variable for m in materias_ids for h in horarios_ids
#             ) <= p.horas_max for d in dias_ids)

#### (3.2.8) limitar dias con clase por docente (NO VA COMO RESTRICCION)


$$ \sum_{m,d} {w_{mp}·v_{md}} \leq {Dmax}_p $$
$$ \sum_{d} {z_{pd}} \leq {Dmax}_p $$

    para todo p


In [395]:
# MAL IMPLEMENTADA

# for p in profesores:
#     if not p.cantidad_dias is None:
#         # mats = [search_materias_by_nombre(materias, m).id for m in p.materias()]
#         model.addConstr(gp.quicksum(
#             w_dict[m, p.id].variable * v_dict[m,d].variable for m in materias_ids for d in dias_ids
#             ) <= p.cantidad_dias)
        

In [396]:
# p = search_profesor_by_nombre(profesores, "jf")
# print(gp.quicksum(
#             w_dict[m, p.id].variable * v_dict[m,d].variable for m in materias_ids for d in dias_ids
#             ).getValue())

### (3.3) Restricciones externas

#### (3.3.1) cantidad de salones

$$ \sum_{m} {u_{mb}} \leq K $$

    para todo b

In [397]:
K = 13      # 13 salones en este caso

def constr_cantidad_salones(model, u_dict):
    model.addConstrs(gp.quicksum(u_dict[m, b].variable for m in materias_ids) <= K for b in bloques_horario_ids)

#### (3.3.2) restricciones ad hoc

In [398]:
def constr_ad_hoc(model, u_dict, v_dict, w_dict, x_dict, y_dict, z_dict):

    # MATERIAS

    ## no dias consecutivos y no coincidir dia para ADA
    # ada = [m.id for m in search_materias_by_nombre(materias, "ADA")]
    # if len(ada) == 2:
    #     model.addConstr(gp.quicksum(v_dict[ada[0],d].variable * v_dict[ada[1],d+1].variable for d in dias_ids[0:-1]) == 0)
    #     model.addConstr(gp.quicksum(v_dict[ada[1],d].variable * v_dict[ada[0],d+1].variable for d in dias_ids[0:-1]) == 0)
    #     model.addConstr(gp.quicksum(v_dict[ada[0],d].variable * v_dict[ada[1],d].variable for d in dias_ids) == 0)

    ## fijar MDp al ultimo bloque (se adiciona horario excepcional)
    # for mdp in search_materias_by_nombre(materias, "MDp"):
    #     if mdp.carga_horaria == 1:
    #         model.addConstr(gp.quicksum(u_dict[mdp.id, (d,h)].variable for d in dias_ids for h in horarios_ids_turno(horarios, "m")[0:-1]) == 0)

    ## coincidir horarios para ambos grupos de SDig
    # sdig = [m.id for m in search_materias_by_nombre(materias, "SDig")]
    # if len(sdig) == 2:
    #     model.addConstrs(u_dict[sdig[0], b].variable == u_dict[sdig[1], b].variable for b in bloques_horario_ids)

    ## coincidir UIUX electiva para 5TEL/INF con obligatoria para 3LIC (se cubre hora excepcional, es decir menor carga horaria para 5to)
    ## fijar a ultimo bloque UIUX electiva
    # try:
    #     uiux_elec = [m.id for m in search_materias_by_nombre(electivas(materias), "UIUX")][0]
    #     model.addConstr(gp.quicksum(u_dict[uiux_elec, (d,h)].variable for d in dias_ids for h in horarios_ids_turno(horarios, "t2")[0:-1]) == 0)
    #     try:
    #         uiux_lic = [m.id for m in search_materias_by_nombre(materias, "UIUX") if "3LIC" in [str(g) for g in m.grupos]][0]
    #         model.addConstrs(u_dict[uiux_lic, b].variable >= u_dict[uiux_elec, b].variable for b in bloques_horario_ids)
    #     except:
    #         pass
    # except:
    #     pass


    # PROFESORES

    ## 2 dias (minimo posible) para dj (coincidir mañana y tarde)
    # dj = search_profesor_by_nombre(profesores, "dj")
    # if dj is not None:
    #     model.addConstr(gp.quicksum(z_dict[dj.id, d].variable for d in dias_ids) == 2)

    
    ## evitar 4 horas seguidas para mp
    # mp = search_profesor_by_nombre(profesores, "mp")
    # if mp is not None:
    #     model.addConstrs(y_dict[mp.id, (d,h)].variable
    #                     + y_dict[mp.id, (d,h+1)].variable
    #                     + y_dict[mp.id, (d,h+2)].variable
    #                     + y_dict[mp.id, (d,h+3)].variable
    #                     <= 3
    #                     for h in horarios_ids[0:-3]
    #                     for d in dias_ids)
    #     # alternativa no lineal:
        # model.addConstr(gp.quicksum(
        #                 y_dict[mp.id, (d,h)].variable
        #                 * y_dict[mp.id, (d,h+1)].variable
        #                 * y_dict[mp.id, (d,h+2)].variable
        #                 * y_dict[mp.id, (d,h+3)].variable
        #                 for h in horarios_ids[0:-3]
        #                 for d in dias_ids)
        #                 == 0)

    ## alternar dias para gb (max 4 dias en total)
    # gb = search_profesor_by_nombre(profesores, "gb")
    # if gb is not None:
    #     model.addConstr(
    #         gp.quicksum(z_dict[gb.id, d].variable for d in dias_ids)
    #         - gp.quicksum(z_dict[gb.id, d].variable * z_dict[gb.id, d+1].variable for d in dias_ids[0:-1])
    #         >= 2)
    
    ## alternar dias para svv
    # svv = search_profesor_by_nombre(profesores, "svv")
    # if svv is not None:
    #     model.addConstr(
    #         gp.quicksum(z_dict[svv.id, d].variable for d in dias_ids)
    #         - gp.quicksum(z_dict[svv.id, d].variable * z_dict[svv.id, d+1].variable for d in dias_ids[0:-1])
    #         >= 2)

    pass
      


#### (3.3.3) practico despues del teorico

$$ \max_b\{(-b)*u_{mb}\} > \max_b\{(-b)*u_{m'b}\} $$

    para todo par (teorico, practico) =  (m, m')


In [399]:
def constr_teo_prac(model, u_dict):

    """
    Variables:
        teo (list): Array containing IDs of theoretical subjects.
        prac (list): Array containing IDs of practical subjects.
        vars1erHora (dict): Dictionary mapping subject IDs to Gurobi integer variables representing the first hour.
        scaled_u_dict (dict): Dictionary mapping tuples of subject IDs and block IDs to scaled Gurobi integer variables.

    Functions:
        search_materias_by_nombre(materias, nombre): Searches for subjects by name in the given list of subjects.

    Workflow:
    1. Populate `teo` and `vars1erHora` with IDs and variables for theoretical subjects.
    2. Populate `prac` and `vars1erHora` with IDs and variables for practical subjects.
    3. Create scaled variables `scaled_u_dict` for each subject and block combination.
    4. Add constraints to the model to ensure that `vars1erHora` variables are the maximum of the scaled variables.
    5. Add constraints to ensure that the first hour of theoretical subjects is greater than or equal to the first hour of practical subjects if the lengths of `teo` and `prac` are equal.


    """
    teo = []    # array con IDS de materias de teorico
    prac = []    # array con IDS de materias de practico
    vars1erHora = {}
    scaled_u_dict = {}

    for t in [m for m in materias if m.teo_prac == "teo"]:
        teo.append(t.id)
        vars1erHora[t.id] = model.addVar(vtype=GRB.INTEGER, name=str(t)+"_1er_hora")

    for p in [m for m in materias if m.teo_prac == "prac"]:
        prac.append(p.id)
        vars1erHora[p.id] = model.addVar(vtype=GRB.INTEGER, name=str(p)+"_1er_hora")

    L = len(bloques_horario_ids)

    for m in teo + prac:
        for b in range(0, L):
            scaled_u_dict[m, bloques_horario_ids[b]] = model.addVar(vtype=GRB.INTEGER, name=str(m)+"scaled_u")
            model.addConstr(scaled_u_dict[m, bloques_horario_ids[b]] == (L - b) * u_dict[m, bloques_horario_ids[b]].variable)

    for m_id in vars1erHora:
        print(m_id, vars1erHora[m_id])
        model.addConstr(vars1erHora[m_id] ==
                        # gp.max_( (L - b) * u_dict[m_id, bloques_horario_ids[b]].variable
                        gp.max_( scaled_u_dict[m_id, bloques_horario_ids[b]]
                                for b in range(0, L)) )

    if len(teo) == len(prac):
        model.addConstrs(vars1erHora[teo[m]] >= vars1erHora[prac[m]] for m in range(0, len(teo)))


### (3.4) Grupos

#### (3.4.1) definicion de variable "x"

$$ x_{gb} = \sum_{m \in M_g} {u_{mb}} $$

$$ x_{gb} = OR_{m \in M_g} {u_{mb}} $$



In [400]:
def constr_definir_x(model, x_dict, u_dict):

    # SUMA
    # for g in grupos:
    #     gr_mats_ids = [m.id for m in materias_grupo(g, materias)]
    #     model.addConstrs(x_dict[g.id, b].variable == gp.quicksum(u_dict[m, b].variable for m in gr_mats_ids) for b in bloques_horario_ids)

    # OR
    for g in grupos:
        # gr_mats_ids = [m.id for m in materias_grupo(g, materias)]
        gr_mats_ids = [m.id for m in materias_grupo(g, [m for m in materias if not m.electiva])] # solo materias obligatorias
        model.addConstrs(x_dict[g.id, b].variable == gp.or_(u_dict[m, b].variable for m in gr_mats_ids) for b in bloques_horario_ids)



#### (3.4.2) evitar horas puente por grupo

$$ \sum_h {x_{gdh}} - \sum_h {x_{gd(h)} · x_{gd(h+1)}} \leq 1 $$
    para todo g, d

In [401]:
def constr_horas_puente_grupos(model, x_dict):

    for g in grupos:
        model.addConstrs(gp.quicksum(x_dict[(g.id,(d,h))].variable for h in horarios_ids)
                    - gp.quicksum(x_dict[(g.id,(d,h))].variable * x_dict[(g.id,(d,h+1))].variable for h in horarios_ids[0:-1])
                    <= 1 for d in dias_ids)

## (4) Funcion Objetivo

### (4.1) Prioridad horaria de los docentes

$$ \min: \sum_{m,b,p} {A_{pb}·w_{mp}·u_{mb}} $$


In [402]:
def obj_prioridades(u_dict, w_dict):

    OBJ1 = gp.QuadExpr()
    """
    This script constructs a quadratic expression for an optimization problem using Gurobi's QuadExpr.
    Variables:
        OBJ1 (gp.QuadExpr): A quadratic expression object to accumulate the terms.
        count (int): A counter to keep track of the number of terms added to the quadratic expression.
    Loops:
        The script iterates over all combinations of 'materias' and 'profesores'. For each professor, it further iterates over their 'prioridades'.
    Inner Loop:
        For each priority 'pr' of a professor:
            - 'b' is assigned the 'bloque_horario' of the priority.
            - 'A' is assigned the 'value' of the priority.
            - The quadratic expression 'OBJ1' is updated by adding the product of 'A', the variable corresponding to the combination of 'materia' and 'profesor' from 'w_dict', and the variable corresponding to the combination of 'materia' and 'bloque_horario' from 'u_dict'.
            - The counter 'count' is incremented.
    Final Step:
        The constructed quadratic expression 'OBJ1' is added to the main objective 'OBJ'.
    Output:
        The total count of terms added to the quadratic expression is printed.
    """

    count = 0
    for m in materias:
        for p in profesores:
            for pr in p.prioridades:
                b = pr.bloque_horario
                A = pr.value
                OBJ1 += A * w_dict[m.id, p.id].variable * u_dict[m.id, b.id].variable
                count += 1


    print("obj_prioridades terms:", count)

    return OBJ1


LINEALIZAR: SUSTITUIR CON VARIABLES y_pb

$$ \min: \sum_{p,b} {A_{pb}·y_{pb}} $$

In [403]:
def obj_prioridades_lineal(y_dict):

    OBJ1 = gp.QuadExpr()

    count = 0
    for p in profesores:
        for pr in p.prioridades:
            b = pr.bloque_horario
            A = pr.value
            OBJ1 += A * y_dict[p.id, b.id].variable
            count += 1

    print("obj_prioridades_lineal terms:", count)

    return OBJ1

### (4.2) Minimizar/Maximizar dias con clase por profesor
$$ \min: \sum_{p,d} {\alpha_p} {z_{pd}}$$
donde $\alpha_p = 1$ si se minimiza, $\alpha_p = -1$ si se maximiza y $\alpha_p = 0$ si es irrelevante

In [404]:
def obj_min_max_dias(z_dict):

    OBJ2 = gp.LinExpr()
    DP = {}

    count = 0
    for p in profesores:
        
        D_p = gp.quicksum(z_dict[p.id,d].variable for d in dias_ids)
        DP[p.id] = D_p

        # filtrar si el profesor prefiere minimizar o maximizar o ninguno
        match p.min_max_dias:
            case "min":
                OBJ2 += D_p
                count += len(dias_ids)
            case "max":
                OBJ2 -= D_p
                count += len(dias_ids)
            case None:
                pass

    print("obj_min_max_dias terms:", count)

    return OBJ2, DP



### (4.3) Minimizar horas puente por grupo (se incluye directamente como restriccion)

$$ \min: -\sum_{g,d,h} {x_{gd(h)}·x_{gd(h+1)}} $$

In [405]:
def obj_horas_puente_grupos(x_dict):
    OBJ3 = gp.QuadExpr()

    count = 0
    for g in grupos:
        for d in dias_ids:
            OBJ3 += - gp.quicksum(x_dict[(g.id,(d,h))].variable * x_dict[(g.id,(d,h+1))].variable for h in horarios_ids[0:-1])
            count += 1

    print("obj_horas_puente_grupos terms:", count)

    return OBJ3


### (4.4) Minimizar superposicion de electivas

$$ \min:  \sum_{b} \sum_{m, m'}{u_{mb} \times SE_{m,m'} \times u_{m'b}} = 0 $$

para todo par (m, m') de electivas

In [406]:
def obj_superposicion_electivas(u_dict):
    OBJ4 = gp.QuadExpr()

    count = 0
    for m1 in electivas(materias):
        for m2 in electivas(materias):
            for b in bloques_horario_ids:
                OBJ4 += u_dict[m1.id, b].variable * superposicion_electivas[m1.id, m2.id].value * u_dict[m2.id, b].variable
                count += 1

    print("obj_superposicion_electivas terms:", count)

    return OBJ4

### (4.5) Evitar horas excepcionales

$$ \min: \sum_{m} \sum_{b \in \mathcal{E}_m} {u_{mb}} $$

In [407]:
def obj_horarios_excepcionales(u_dict):

    objetivo = gp.QuadExpr()

    count = 0
    for m in materias:
        for b in bloques_horario.values():
            excep = b.horario.turnos_excepcional
            if len(intersection(excep, m.turnos()))>0:
                objetivo += u_dict[m.id, b.id].variable
                count += 1
        
    print("obj_horarios_excepcionales terms:", count)

    return objetivo

# Resolver

### Filtrar (ejecutar desde acá)

In [408]:
# recargar input original

import copy

anios = copy.deepcopy(ANIOS)
dias = copy.deepcopy(DIAS)
horarios = copy.deepcopy(HORARIOS)
bloques_horario = copy.deepcopy(BLOQUES_HORARIO)
profesores = copy.deepcopy(PROFESORES)
grupos = copy.deepcopy(GRUPOS)
materias = copy.deepcopy(MATERIAS)
superposicion = copy.deepcopy(SUPERPOSICION)
superposicion_electivas = copy.deepcopy(SUPERPOSICION_ELECTIVAS)

In [409]:
#filtrar por grupos

grupos: list[Grupo] = [g for g in grupos if g.anio in [2] and g.carrera not in ["IND"]]

# materias = [m for m in materias if str(m) not in ["tAM1"]]

ms = []
for m in materias:
    gs_m = []
    for g in grupos:
        if g in m.grupos:
            gs_m.append(g)

    if len(gs_m) > 0:
        m.grupos = gs_m
        ms.append(m)

materias: list[Materia] = ms


# ajustar conjunto de profesores y grupos segun las materias filtradas

ps = []
gs = []

for m in materias:
    # print(str(m), ":", [str(g) for g in m.grupos], [str(p) for p in m.profesores], m.carga_horaria, m.cantidad_dias)
    for p in m.profesores:
        if p not in ps:
            ps.append(p)
    for g in m.grupos:
        if g not in gs:
            gs.append(g)

for p in ps: # ajustar materias que puede dictar un profesor
    p.lista_materias = [m for m in p.lista_materias if m['nombre_materia'] in [str(m) for m in materias]]
    # print(f"{p}: {set([str(m) for m in materias if p in m.profesores])} =? {[f'{m['nombre_materia'], m['grupos_max']}' for m in p.lista_materias]}")

grupos: list[Grupo] = gs
profesores: list[Profesor] = ps

anios = [a for a in anios if a in [g.anio for g in grupos]]


In [410]:
print([str(g) for g in grupos])

['2CIV', '2TEL', '2DIA', '2INF', '2LINF']


In [411]:
# chequeo de disponibilidad

# for p in profesores:
#     print(p, p.nombre_completo)
#     for l in p.lista_materias:
#         for mat in search_materias_by_nombre(materias, l["nombre_materia"]):
#             if p not in mat.profesores:
#                 continue
#             print('\t', l["nombre_materia"], l["grupos_max"], mat.carga_horaria)
#             print('\t\thoras:', mat.carga_horaria*l["grupos_max"])
            
#             disp = bloques_horario.values()
#             for t in mat.turnos():
#                 disp = [d for d in disp if t in d.horario.turnos]
            
#             disp = [d for d in disp if d not in p.no_disponible]
#             print('\t\thoras disponibles:', len(disp))
        


#### Superposicion de electivas

In [412]:

# # BPMN y dbdIII coincidan horarios
# fix_super(0, superposicion_electivas, search_materias_by_nombre(electivas(materias), "BPMN"), search_materias_by_nombre(electivas(materias), "DBD3"))

# # RPA y aplicaciones móviles coincidan horarios
# fix_super(0, superposicion_electivas, search_materias_by_nombre(electivas(materias), "RPA"), search_materias_by_nombre(electivas(materias), "ApMov"))

# # Diseño estructural y Pavimentos coincidan horarios
# fix_super(0, superposicion_electivas, search_materias_by_nombre(electivas(materias), "DisEst"), search_materias_by_nombre(electivas(materias), "DPav"))

# # Diseño estructural + Puentes pueden coincidir con Diseño de Pavimentos + Diseño de infraestructura pluvial y urbana
# fix_super(0, superposicion_electivas,
#             [m for m in electivas(materias) if m.nombre in ["DisEst", "Puentes"]],
#             [m for m in electivas(materias) if m.nombre in ["DPav", "DIPyU"]]
#             )



In [413]:
# fix_super(1, superposicion,
#             search_materias_by_nombre(electivas(materias), "DBD3"),
#             search_materias_by_nombre(electivas(materias), "ApMov"))


# # ---------------- 5TEL / 5INF / 4LIC ----------------

# bloque1 = [m for m in electivas(materias) if m.nombre in ["RPA", "BPMN", "GSInf"]]
# bloque2 = [m for m in electivas(materias) if m.nombre in ["ApMov", "DBD3", "TestQA"]]
# bloque3 = [m for m in electivas(materias) if m.nombre in ["QC", "EthHck", "UIUX"]]

# fix_super(0, superposicion, bloque1, bloque2)
# fix_super(0, superposicion, bloque2, bloque3)
# fix_super(0, superposicion, bloque3, bloque1)

# # ---------------------- 5CIV ------------------------

# fix_super(0, superposicion,
#             search_materias_by_nombre(electivas(materias), "DPav"),
#             search_materias_by_nombre(electivas(materias), "DIPyU"))
            
# fix_super(0, superposicion, search_materias_by_nombre(electivas(materias), "Puentes")
#             # ,electivas([m for m in materias if m.nombre != "DPav"])
#             ,electivas(materias)
#             )


In [414]:
# for g in grupos:
#     ms = [e for e in electivas(materias) if g in [g for g in e.grupos]]
#     if len(ms) > 0:
#         print(g)
#     for m1 in ms:
#         print('\t', m1)
#         for m2 in [m for m in ms if m1 != m]:
#             print('\t\t', m2, superposicion[(m1.id, m2.id)].value)

In [415]:
# actualizar id arrays

dias_ids = [d.id for d in dias]

horarios_ids = [h.id for h in horarios]

bloques_horario_ids = [b for b in bloques_horario]

profesores_ids = [p.id for p in profesores]

grupos_ids = [g.id for g in grupos]

materias_ids = [m.id for m in materias]


## Chequeos previos

In [416]:
# # chequeos de consistencia para asignacion de profesores y materias

nombres_materias = []
for m in materias:
    if str(m) not in nombres_materias:
        nombres_materias.append(str(m))

count_asignaciones = {}
for m in nombres_materias:
    count_asignaciones[m] = 0

count = 0
for m in nombres_materias:
    length = len(search_materias_by_nombre(materias, m))
    # print(m, length)
    count_profs = 0
    for mi in search_materias_by_nombre(materias, m):
        count_profs += mi.cantidad_profesores
    count_asignaciones[m] += count_profs
    count += count_profs

print(count)

count2 = 0
for p in profesores:
    # print(p, p.nombre_completo)
    for l in p.lista_materias:
        # print('\t', l["nombre_materia"], l["grupos_max"], len([m for m in search_materias_by_nombre(materias, l["nombre_materia"]) if p in m.profesores]))
        count2 += l["grupos_max"]
        count_asignaciones[l["nombre_materia"]] -= l["grupos_max"]
        # for m in search_materias_by_nombre(materias, l["nombre_materia"]):
            # print(len(mi for mi in))
            # print('\t\t', m, *m.profesores)
    # print(p, *[str(l["nombre_materia"]) + ": " + str(l["grupos_max"]) for l in p.lista_materias])

print(count2)

print(*[c + ": " + str(count_asignaciones[c]) for c in count_asignaciones], sep="\n")


30
30
Prob: 0
ProbP: 0
ADA: 0
ADAp: 0
TIC1: 0
RD1: 0
TCirc: 0
TComp: 0
Mec2: 0
RM1: 0
PC1: 0
PC/O1: 0
HC: 0
SO: 0
IO: 0
ISW1: 0
DBD2: 0
InProb: 0


In [417]:
# chequeo de carga horaria por grupo

print(f"Grupo: \t\thoras <= bloques \telectivas")
print("--------------------------------------------------")
for g in grupos:
    print(f"{g}: \t\t{np.sum([m.carga_horaria for m in materias_grupo(g, materias)])} <= {len([b for b in bloques_horario.values() if g.turno in b.horario.turnos])} \t\t{np.sum([m.carga_horaria for m in materias_grupo(g, materias) if m.electiva])}")
    

Grupo: 		horas <= bloques 	electivas
--------------------------------------------------
2CIV: 		32 <= 35 		0.0
2TEL: 		29 <= 35 		0.0
2DIA: 		27 <= 35 		0.0
2INF: 		27 <= 35 		0.0
2LINF: 		23 <= 30 		0.0


## Compilar modelo

In [418]:
# Create a new model
model: gp.Model = gp.Model("timetable")

In [419]:
# variables
u_dict, v_dict, w_dict, x_dict, y_dict, z_dict = initialize_variables(materias, bloques_horario, dias, profesores, grupos)
create_variables(model, u_dict, v_dict, w_dict, x_dict, y_dict, z_dict)

u:  2320
v:  145
w:  638
x:  400
y:  1760
z:  110


### Seleccionar restricciones

In [420]:
constr_superposicion(model, u_dict)
constr_carga_horaria(model, u_dict)
constr_dias_materia(model, v_dict)
constr_max_min_horas(model, u_dict, v_dict)

In [421]:
constr_turnos_materia(model, u_dict)
constr_horas_consecutivas(model, u_dict, v_dict)
constr_dias_consecutivos(model, v_dict)
# constr_no_disponible_materia(model, u_dict)

In [422]:
constr_no_disponible_profesor(model, u_dict, w_dict)
# constr_no_disponible_profesor_lineal(model, y_dict)
constr_unica_materia_profesor(model, u_dict, w_dict)
constr_limitar_profesores_materia(model, w_dict)
constr_cantidad_profesores(model, w_dict)
constr_grupos_max_profesor(model, w_dict)

In [423]:
constr_definir_y(model, y_dict, u_dict, w_dict)
constr_definir_z(model, z_dict, y_dict)

In [424]:
constr_cantidad_salones(model, u_dict)
constr_ad_hoc(model, u_dict, v_dict, w_dict, x_dict, y_dict, z_dict)
constr_teo_prac(model, u_dict)

36 <gurobi.Var *Awaiting Model Update*>
37 <gurobi.Var *Awaiting Model Update*>
38 <gurobi.Var *Awaiting Model Update*>
42 <gurobi.Var *Awaiting Model Update*>
43 <gurobi.Var *Awaiting Model Update*>
39 <gurobi.Var *Awaiting Model Update*>
40 <gurobi.Var *Awaiting Model Update*>
41 <gurobi.Var *Awaiting Model Update*>
44 <gurobi.Var *Awaiting Model Update*>
45 <gurobi.Var *Awaiting Model Update*>


In [425]:
constr_definir_x(model, x_dict, u_dict)
constr_horas_puente_grupos(model, x_dict)

### Funcion objetivo 

In [426]:
# funcion objetivo

objectives = { # name : (expr, weight)
    
    "prioridades" : (obj_prioridades(u_dict, w_dict), 1),
    # "prioridades_lineal" : (obj_prioridades_lineal(y_dict), 1),
    "min_max_dias" : (obj_min_max_dias(z_dict)[0], 5),
    # ("horas_puente_grupos", obj_horas_puente_grupos(x_dict), 1),
    "superposicion_electivas" : (obj_superposicion_electivas(u_dict), 5),
    "horarios_excepcionales" : (obj_horarios_excepcionales(u_dict), 100),

}

OBJ = gp.QuadExpr()
for name in objectives:
    expr, weight = objectives[name]
    OBJ += weight * expr

model.setObjective(OBJ, GRB.MINIMIZE)

obj_prioridades terms: 51040
obj_min_max_dias terms: 0
obj_superposicion_electivas terms: 0
obj_horarios_excepcionales terms: 145


## Optimizar

In [427]:
model.setParam("TimeLimit", 60*60*2)
# model.Params.MIPGap = 1/100
# model.Params.PoolSolutions = 10 # Number of solutions to find
# model.Params.PoolSearchMode = 0 # 0, 1, 2

model.optimize()

# nSolutions = model.SolCount
# print(f"Number of solutions found: {nSolutions}")

Set parameter TimeLimit to value 7200
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3212 rows, 9543 columns and 13931 nonzeros
Model fingerprint: 0x2915da3f
Model has 51040 quadratic objective terms
Model has 5399 quadratic constraints
Model has 2280 general constraints
Variable types: 0 continuous, 9543 integer (5373 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  QMatrix range    [1e+00, 2e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+02, 1e+02]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
  QRHS range       [1e+00, 1e+00]
Presolve removed 1495 rows and 7412 columns
Presolve time: 0.14s
Presolved: 6507 rows, 3489 columns, 19638 nonzeros
Variable types: 0 continuous, 3489 in

In [428]:
# # Function to get variable values for a specific solution
# def get_solution_values(model, solution_number):
#     model.setParam("SolutionNumber", solution_number)
#     return {var.varName: round(var.x) for var in model.getVars()}

# # Get the number of solutions
# num_solutions = model.SolCount

# # Store variable values for each solution
# solutions = []
# for i in range(num_solutions):
#     solutions.append(get_solution_values(model, i))

# # Compare solutions to find differences
# differences = {}
# for var_name in solutions[0].keys():
#     values = [solution[var_name] for solution in solutions]
#     if len(set(values)) > 1:  # If there are different values for this variable
#         differences[var_name] = values

# # Print differences
# for var_name, values in differences.items():
#     print(f"Variable {var_name} has different values: {values}")

### Analizar no-factibilidad 

In [429]:
if model.Status == GRB.INFEASIBLE:
    model.computeIIS()
    model.write('iismodel.ilp')

    # Print out the IIS constraints and variables
    print('\nThe following constraints and variables are in the IIS:')
    for c in model.getConstrs():
        if c.IISConstr: print(f'\t{c.constrname}: {model.getRow(c)} {c.Sense} {c.RHS}')

    for v in model.getVars():
        if v.IISLB: print(f'\t{v.varname} ≥ {v.LB}')
        if v.IISUB: print(f'\t{v.varname} ≤ {v.UB}')

## Imprimir horarios

In [457]:
model.setParam("SolutionNumber", 0)
model.update()


if not model.Status == GRB.INFEASIBLE:
    print('Obj: %g' % model.ObjVal)
    for name in objectives:
        expr, weight = objectives[name]
        print(f"\t{name}: {" "*(28-len(name))}{round(expr.getValue())} \t(peso={weight})")
    print('=============================================================')
    
    print_timetable(dias, horarios, u_dict, w_dict, grupos, anios)

    copy_variables_excel(u_dict, w_dict, output)

Obj: 323
	prioridades:                  123 	(peso=1)
	min_max_dias:                 0 	(peso=5)
	superposicion_electivas:      0 	(peso=5)
	horarios_excepcionales:       2 	(peso=100)

 -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
Año:  2

 Grupo:  2CIV
			lun		mar		mie		jue		vie
··········································································································
08:00-08:50		Mec2		PC/O1		Prob		RM1		PC1
08:50-09:40		Mec2		PC/O1		Prob		RM1		PC1
09:50-10:40		Prob		PC/O1		PC1		RM1		PC1
10:40-11:30		Prob		PC/O1		PC1		Mec2		HC
11:40-12:30		RM1		PC/O1		HC		Mec2		HC
12:30-13:20		RM1		PC/O1		HC		Mec2		ProbP
13:20-14:10		RM1		---		---		---		ProbP

 Grupo:  2TEL
			lun		mar		mie		jue		vie
··········································································································
08:00-08:50		RD1		HC		---		RD1		ADAp
08:50-09:40		RD1		HC		Prob		RD1		ADAp
09:50-10:40		RD1		TIC1		Prob		ProbP		TIC1
10:40-11:

In [431]:
# if not model.Status == GRB.INFEASIBLE:
#     # print_prof_timetable(dias, horarios, u_dict, w_dict, profesores, materias)
#     # print_prof_timetable(dias, horarios, u_dict, w_dict, profesores, search_materias_by_nombre(materias, "ADA"))

#     # print_prof_timetable_excel(dias, horarios, u_dict, w_dict, profesores, materias)

In [432]:
# imprimir calendario para profesores
# if not model.Status == GRB.INFEASIBLE:
    # print_prof_timetable(dias, horarios, u_dict, w_dict, [search_profesor_by_nombre(profesores, "gc")], materias)
    # print_prof_timetable(dias, horarios, u_dict, w_dict, profesores, search_materias_by_nombre(materias,"Prob"))
    # print_prof_timetable(dias, horarios, u_dict, w_dict, profesores, materias)

#### Imprimir para Excel

In [433]:
# # # IMPRIMIR PARA COPIAR EN HOJA DE EXCEL
if not model.Status == GRB.INFEASIBLE:
    print_timetable_excel(dias, horarios, u_dict, w_dict, grupos, anios, show_profs=True)


 -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
Año:  2

 Grupo:  2CIV
	lun	mar	mie	jue	vie
08:00-08:50	Mec2[cb]	PC/O1[nd]	Prob[jd]	RM1[as]	PC1[bt]
08:50-09:40	Mec2[cb]	PC/O1[nd]	Prob[jd]	RM1[as]	PC1[bt]
09:50-10:40	Prob[jd]	PC/O1[nd]	PC1[bt]	RM1[as]	PC1[bt]
10:40-11:30	Prob[jd]	PC/O1[nd]	PC1[bt]	Mec2[cb]	HC[]
11:40-12:30	RM1[as]	PC/O1[nd]	HC[]	Mec2[cb]	HC[]
12:30-13:20	RM1[as]	PC/O1[nd]	HC[]	Mec2[cb]	ProbP[ds]
13:20-14:10	RM1[as]	---	---	---	ProbP[ds]

 Grupo:  2TEL
	lun	mar	mie	jue	vie
08:00-08:50	RD1[gco,mjc,sb]	HC[]	---	RD1[gco,mjc,sb]	ADAp[jcc]
08:50-09:40	RD1[gco,mjc,sb]	HC[]	Prob[jk]	RD1[gco,mjc,sb]	ADAp[jcc]
09:50-10:40	RD1[gco,mjc,sb]	TIC1[nn]	Prob[jk]	ProbP[ds]	TIC1[nn]
10:40-11:30	TCirc[ma]	TIC1[nn]	TCirc[ma]	ProbP[ds]	TIC1[nn]
11:40-12:30	TCirc[ma]	ADA[jcc]	TCirc[ma]	HC[]	Prob[jk]
12:30-13:20	TCirc[ma]	ADA[jcc]	TCirc[ma]	HC[]	Prob[jk]
13:20-14:10	---	---	---	---	---

 Grupo:  2DIA
	lun	mar	mie	jue	vie
08:00-08:50	RD1[gc

In [434]:
if not model.Status == GRB.INFEASIBLE:
    print_prof_timetable_excel(dias, horarios, u_dict, w_dict, profesores, materias)


 Profesor:  jd José Díaz
	lun	mar	mie	jue	vie
08:00-08:50	---	---	Prob	---	---
08:50-09:40	---	---	Prob	---	---
09:50-10:40	Prob	---	---	---	---
10:40-11:30	Prob	---	---	---	---

 Profesor:  jk Juan Kalemkerian
	lun	mar	mie	jue	vie
08:50-09:40	---	---	Prob	---	---
09:50-10:40	---	---	Prob	---	---
11:40-12:30	---	---	---	---	Prob
12:30-13:20	---	---	---	---	Prob

 Profesor:  mpe Mariana Pereira
	lun	mar	mie	jue	vie
08:00-08:50	---	---	Prob	---	---
08:50-09:40	---	---	Prob	---	---
09:50-10:40	---	---	---	---	Prob
10:40-11:30	---	---	---	---	Prob

 Profesor:  ds Débora Stalker
	lun	mar	mie	jue	vie
08:00-08:50	---	---	---	ProbP	---
08:50-09:40	---	---	---	ProbP	---
09:50-10:40	---	---	---	ProbP	---
10:40-11:30	---	---	---	ProbP	---
12:30-13:20	---	---	---	---	ProbP
13:20-14:10	---	---	---	---	ProbP

 Profesor:  jcc Juan Cruz Carrau
	lun	mar	mie	jue	vie
08:00-08:50	---	---	---	---	ADAp
08:50-09:40	---	---	---	---	ADAp
11:40-12:30	---	ADA	---	---	---
12:30-13:20	---	ADA	---	---	---

 Profes

## Estadísticas de resultados

### Promedios de prioridad

In [435]:
# CARGA_HORARIA_TOTAL = gp.quicksum(m.carga_horaria for m in materias)
CARGA_HORARIA_TOTAL = gp.quicksum(u_dict[m, b].variable * w_dict[m, p].variable for p in profesores_ids for m in materias_ids for b in bloques_horario_ids)

if not model.Status == GRB.INFEASIBLE:
    for obj, _ in [objectives["prioridades"]]:
        print(obj.getValue())
        print("Promedio general de prioridad horaria: ", obj.getValue()/CARGA_HORARIA_TOTAL.getValue())
          

# print("Casos de materias con dias consecutivos: ", int(OBJ2.getValue()/10))


123.0
Promedio general de prioridad horaria:  1.0


In [436]:
# promedio por profesor:
"""
This script calculates and prints the average workload per professor based on their assigned priorities and time blocks.
Variables:
    profesores (list): List of professor objects, each containing their priorities.
    y_dict (dict): Dictionary mapping professor IDs and time block IDs to Gurobi variables.
    bloques_horario_ids (list): List of time block IDs.
For each professor in the list `profesores`, the script performs the following steps:
1. Initializes a quadratic expression `OBJ_p` to accumulate the weighted priorities.
2. Iterates over the professor's priorities to update `OBJ_p` based on the priority value and corresponding Gurobi variable.
3. Calculates the total workload `CARGA_HORARIA_p` for the professor by summing the relevant Gurobi variables.
4. If the total workload is non-zero, it prints the professor's name, total workload, and the rounded average priority value per workload unit.
5. If the total workload is zero, it prints the professor's name and indicates that they have no workload.

"""
if not model.Status == GRB.INFEASIBLE:
    for p in profesores:

        OBJ_p = gp.QuadExpr()
        # for m in materias:
        #     for pr in p.prioridades:
        #             b = pr.bloque_horario
        #             A = pr.value
        #             OBJ_p += A * w_dict[m.id, p.id].variable * u_dict[m.id, b.id].variable

        for pr in p.prioridades:
            b = pr.bloque_horario
            A = pr.value
            OBJ_p += A * y_dict[p.id, b.id].variable
        
        # CARGA_HORARIA_p = gp.quicksum(u_dict[m, b].variable * w_dict[m, p.id].variable for m in materias_ids for b in bloques_horario_ids).getValue()
        CARGA_HORARIA_p = round(gp.quicksum(y_dict[p.id, b].variable for b in bloques_horario_ids).getValue())

        if CARGA_HORARIA_p != 0:
            print(str(p), "\t", CARGA_HORARIA_p, "\t", round(round(OBJ_p.getValue())/CARGA_HORARIA_p,3))
        else:
            print(str(p), "sin carga horaria")

    


jd 	 4 	 1.0
jk 	 4 	 1.0
mpe 	 4 	 1.0
ds 	 6 	 1.0
jcc 	 4 	 1.0
mbo 	 4 	 1.0
nn 	 8 	 1.0
gco 	 10 	 1.0
mjc 	 10 	 1.0
sb 	 10 	 1.0
ma 	 6 	 1.0
dc 	 4 	 1.0
mf 	 4 	 1.0
cb 	 5 	 1.0
as 	 6 	 1.0
bt 	 5 	 1.0
nd 	 6 	 1.0
hr 	 5 	 1.0
ab 	 4 	 1.0
apa 	 4 	 1.0
sgp 	 5 	 1.0
cp 	 5 	 1.0


In [437]:
# # DIAS CLASE POR PROFESOR

# print("DIAS CON CLASE")
# for p in profesores:
#     if p.min_max_dias is not None:
#         print(str(p), ": ", round(DP[p.id].getValue()), p.min_max_dias)

In [438]:
# PROFESORES CON DISPONIBILIDAD LIMITADA

# for p in profesores:
        
#     suma_pr = 0
#     carga_horaria = 0
    
#     for pr in p.prioridades:
#         suma_pr = suma_pr + 1 if pr.value > 0 else suma_pr

#     for m in p.lista_materias:
#         try:
#             horas = search_materias_by_nombre(materias, m["nombre_materia"])[0].carga_horaria
#             grupos_max = m["grupos_max"]
#             carga_horaria += horas*grupos_max
#         except:
#             continue

#     if carga_horaria > 0:
#         margen = int((suma_pr/carga_horaria-1)*100)

#         if margen < 250:
#             print('----------------------')
#             print(str(p), p.nombre_completo)
#             print("disponibilidad:", suma_pr)
#             print("carga horaria:", carga_horaria)
#             print("margen:", margen, "%")




In [439]:
# HORAS PUENTE

# HP = gp.QuadExpr()
# for g in grupos_reales:
#     HPg = gp.QuadExpr()
#     for d in dias_ids:
#         HPg += gp.quicksum(x_dict[(g.id,(d,h))].variable for h in horarios_ids)
#         HPg += - gp.quicksum(x_dict[(g.id,(d,h))].variable * x_dict[(g.id,(d,h+1))].variable for h in horarios_ids[0:-1])
#         HPg += - 1
#     # print("horas puente grupo", str(g), ": ", HPg.getValue())
#     HP += HPg

# print("Total de horas puente: ", HP.getValue())

#### Distribucion de prioridades

In [440]:
# distribucion de prioridades
"""
This script calculates and prints the distribution of priorities and their averages for a given set of professors and their time block priorities.
Variables:
    promedios_prioridad (dict): A dictionary to store the average priority values for each priority level.
    valores_prioridad (dict): A dictionary to store the count of each priority level.
    profesores (list): A list of professor objects, each containing their priority information.
    y_dict (dict): A dictionary containing the decision variables for the optimization model.
Process:
1. Initialize dictionaries to store priority averages and counts.
2. Iterate over each professor and their priorities:
    - Skip if the priority value is 0.
    - Increment the count for the given priority value.
    - Update the average priority value based on the decision variable.
3. Print the count and percentage distribution of each priority level.
4. Print the average priority values and their percentage distribution.

"""

if not model.Status == GRB.INFEASIBLE:
    promedios_prioridad = {}
    valores_prioridad = {}
    for i in range(1,4):
        promedios_prioridad[i] = 0
        valores_prioridad[i] = 0

    for p in profesores:
        for pr in p.prioridades:
            b = pr.bloque_horario
            A = pr.value
            if A == 0:
                continue
            valores_prioridad[A] += 1
            # for m in materias:
            #     promedios_prioridad[A] += round(w_dict[m.id, p.id].variable.X * u_dict[m.id, b.id].variable.X)
            promedios_prioridad[A] += round(y_dict[p.id, b.id].variable.X)

    print(valores_prioridad)
    total = np.sum([valores_prioridad[i] for i in valores_prioridad])
    print(*[str(i) + ": " + str(round(valores_prioridad[i]/total*100))+"%" for i in valores_prioridad], sep=", ")

    print(promedios_prioridad)
    total = np.sum([promedios_prioridad[i] for i in promedios_prioridad])
    if total != 0:
        print(*[str(i) + ": " + str(round(promedios_prioridad[i]/total*100))+"%" for i in promedios_prioridad], sep=", ")
        
        # CARGA_HORARIA_p = gp.quicksum(u_dict[m, b].variable * w_dict[m, p.id].variable for m in materias_ids for b in bloques_horario_ids).getValue()

        # if CARGA_HORARIA_p != 0:
        #     print(str(p), ": ", OBJ_p.getValue()/CARGA_HORARIA_p)
        # else:
        #     print(str(p), "sin carga horaria")

{1: 1760, 2: 0, 3: 0}
1: 100%, 2: 0%, 3: 0%
{1: 123, 2: 0, 3: 0}
1: 100%, 2: 0%, 3: 0%


### Resultados superposicion de electivas

In [441]:
if not model.Status == GRB.INFEASIBLE:
    res_super_electivas = {}

    for g in grupos:
        ms = [e for e in electivas(materias) if g in [g for g in e.grupos]]
        if len(ms) > 0:
            print(g)
            res_super_electivas[g.id] = {}
        for m1 in ms:
            # print('\t', m1)
            for m2 in [m for m in ms if m1.nombre != m.nombre]:
                suma = np.sum([round(u_dict[m1.id, b].variable.x) * round(u_dict[m2.id, b].variable.x) for b in bloques_horario_ids])
                # print('\t\t', m2, sum)
                if (m1.nombre, m2.nombre) not in res_super_electivas[g.id]:
                    res_super_electivas[g.id][m1.nombre, m2.nombre] = suma
                else:
                    res_super_electivas[g.id][m1.nombre, m2.nombre] += suma
        
        nombres = set(m.nombre for m in ms)
        for m1 in nombres:
            print('\t', m1)
            for m2 in [m for m in nombres if m1 != m]:
                print('\t\t', m2, res_super_electivas[g.id][m1, m2])


#### Conjuntos de electivas compatibles

In [442]:
# # CONJUNTOS DE ELECTIVAS COMPATIBLES

# import itertools

# def find_subsets(ms, res_super_electivas_g):
#     # Generate all possible subsets of the array ms
#     subsets = []
#     for r in range(1, len(ms) + 1):
#         subsets.extend(list(itertools.combinations(ms, r)))
    
#     # Filter subsets to only include those with dictionary value 0 between all of them
#     valid_subsets = []
#     for subset in subsets:
#         valid = True
#         for m1, m2 in itertools.combinations(subset, 2):
#             if res_super_electivas_g.get((m1, m2), None) != 0:
#                 valid = False
#                 break
#         if valid:
#             valid_subsets.append(subset)
    
#     return valid_subsets

# # for g in grupos:
# #     ms = set(e.nombre for e in elecs if g in [g for g in e.grupos])
# #     if len(ms) > 0:
# #         print(g)
# #         for s in find_subsets(ms, res_super_electivas[g.id]):
# #             print('\t', [m for m in s])

# for g in grupos:
#     ms = set(e.nombre for e in electivas(materias) if g in [g for g in e.grupos])
#     if len(ms) > 0:
#         print(g)
#         subsets = find_subsets(ms, res_super_electivas[g.id])
#         for m in ms:
#             s_max = [m]
#             for s in [s for s in subsets if m in s]:
#                 if len(s) > len(s_max):
#                     s_max = s
#             print('\t', m, [mi for mi in s if mi != m])

#         # for s in subsets:
#         #     print('\t', [m for m in s])


In [443]:
# def include_mat(set, m):
#     include = True
#     for mprev in set:
#         if m != mprev and res_super_electivas[m.id, mprev.id] != 0:
#             include = False
#             break
#     return include

# def update_set(set, ms):
#     sets = []
    
#     if len(ms) == 0:
#         return [set]
    
#     # if len(ms) == 1:
#     for mi in ms:
#         if include_mat(set, mi):
#             sets += update_set(set + [mi], ms[1:])
#             print(*ms[1:])
#         else:
#             sets += set

#     # else:
#     #     if include_mat(set, mi):
#     #         sets += update_set(set + [m1], ms[1:])
#     #     sets += update_set(set, ms[1:])                
#     return sets
        
        
# for g in grupos:
#     ms = [e for e in elecs if g in [g for g in e.grupos]]
#     for m1 in ms:
#         sets_posibles += update_set([m1], [m for m in ms if m != m1])


#     # update_set(sets_posibles, ms)



### Ocupacion de salones

In [444]:
# nivel de ocupacion
if not model.Status == GRB.INFEASIBLE:
    niveles = {}

    for b_id in bloques_horario:
        salones_ocupados = int(gp.quicksum(u_dict[m, b_id].variable for m in materias_ids).getValue())
        nivel = int(salones_ocupados/K*100)
        # print(str(bloques_horario[b_id]), ": ", salones_ocupados,",", str(nivel) + "%")
        niveles[b_id] = nivel

    print_timetable_salones(dias, horarios, niveles)

		lun	mar	mie	jue	vie
··········································································································
08:00-08:50	15%	15%	15%	23%	23%
08:50-09:40	23%	23%	23%	23%	23%
09:50-10:40	23%	23%	23%	23%	23%
10:40-11:30	30%	23%	30%	23%	23%
11:40-12:30	30%	23%	30%	23%	23%
12:30-13:20	23%	15%	23%	23%	23%
13:20-14:10	7%	0%	0%	0%	7%
14:10-15:00	0%	0%	0%	0%	0%
15:10-16:00	0%	0%	0%	0%	0%
16:10-17:00	0%	0%	0%	0%	0%
17:00-17:50	0%	0%	0%	0%	0%
18:00-18:50	7%	7%	7%	7%	0%
18:50-19:40	7%	7%	7%	7%	7%
19:50-20:40	7%	7%	7%	7%	7%
20:40-21:30	7%	7%	7%	7%	7%
21:40-22:30	7%	7%	0%	7%	7%


In [445]:
model.write('timetable_opt.lp')
model.write('timetable_opt.mps')
